In [ ]:
fin = open("superposed_structures.txt", "rt")
fout = open("superposed_structures_.txt", "wt")

for line in fin:
    
    fout.write(' '.join(line.split()))
    fout.write('\n')

fin.close()
fout.close()

In [4]:
import numpy as np
from scipy.linalg import svd
from scipy.linalg import det
from scipy.linalg import norm
import math
import matplotlib.pyplot as plt

fasta_file = open('pdbefold_final_fasta.txt','r')
coordinates_file = open('superposed_structures_.txt','r')
data = fasta_file.read()
coordinates = coordinates_file.read()

alignment_list=data.split('\n\n')
coordinates_list=coordinates.split('\n\n')

alignment_dict={}
coordinates_dict={}

for entry in coordinates_list:
    
    coordinates_values = entry.split('\n')
    coordinates_dict[coordinates_values[0]] = np.zeros((1000, 3))
    index = 0
    for line in coordinates_values:
        if index > 0:
            xyz = line.split(' ')
            coordinates_dict[coordinates_values[0]][index-1][0] = xyz[0]
            coordinates_dict[coordinates_values[0]][index-1][1] = xyz[1]
            coordinates_dict[coordinates_values[0]][index-1][2] = xyz[2]
        index+=1

for entry in alignment_list:
    start = 0
    started = 0
    name = ''
    for char in entry:
        
        if start == 1:
            name=name+char
        if char == ':' and started == 0:
            start = 1
            started = 1
        elif char == ':' and started == 1:
            start = 0
            started = 0

    name = name[:-1]
    alignment_info = ''
    for char in entry:
        if start == 1:
            alignment_info = alignment_info+char
        if char == '\n':
            start = 1

    alignment_dict[name] = [alignment_info, 0, 0]

for key in alignment_dict:
    found_start = 0
    for index in range(len(alignment_dict[key][0])):
        if alignment_dict[key][0][index] != '\n' and alignment_dict[key][0][index]!='-' and found_start == 0:
            found_start = 1
            alignment_dict[key][1] = index

coordinates_matrix = np.zeros((1000, 4))

coordinates_matrix_index = 0
for index in range(len(alignment_dict['4aaj'][0])):
    match = 0
    empty_space = 0
    for key in alignment_dict:
        
        if alignment_dict[key][0][index] != '\n' and alignment_dict[key][0][index]!='-':
            alignment_dict[key][2]+=1
            match +=1
            m_index = alignment_dict[key][2]
            coordinates_matrix[coordinates_matrix_index][0]+=coordinates_dict[key][m_index-1][0]
            coordinates_matrix[coordinates_matrix_index][1]+=coordinates_dict[key][m_index-1][1]
            coordinates_matrix[coordinates_matrix_index][2]+=coordinates_dict[key][m_index-1][2]
            coordinates_matrix[coordinates_matrix_index][3]+=1
        else:
            empty_space+=1

    if empty_space < len(alignment_dict) :
        coordinates_matrix_index+=1

index_zero = 0
for index in range(len(coordinates_matrix)):
    if coordinates_matrix[index][0] == 0 and coordinates_matrix[index][1] == 0 and coordinates_matrix[index][2] == 0:
        index_zero = index
        break
        
coordinates_matrix = coordinates_matrix[0:index_zero]

for index in range(len(coordinates_matrix)):
    coordinates_matrix[index][0] = coordinates_matrix[index][0]/coordinates_matrix[index][3]
    coordinates_matrix[index][1] = coordinates_matrix[index][1]/coordinates_matrix[index][3]
    coordinates_matrix[index][2] = coordinates_matrix[index][2]/coordinates_matrix[index][3]

coordinates_matrix = np.delete(coordinates_matrix, -1, axis=1)

np.savetxt('average_structure_coordinates.txt', coordinates_matrix, fmt='%.4f')

In [6]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import numpy as np
%matplotlib
 
x = coordinates_matrix[:,0]
    
y = coordinates_matrix[:,1]
z = coordinates_matrix[:,2]
# Creating figure
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
 
# Creating plot
ax.scatter3D(x, y, z, color = "green")
plt.title("simple 3D scatter plot")
 
# show plot
plt.show()

Using matplotlib backend: MacOSX
